In [0]:
## This Cell is related to date preprocessing and is completely copied from original kipf's github code!

import numpy as np
import pandas as pd
import scipy.sparse as sp
import torch,torch.nn,torch.sparse,torch.nn.functional,torch.distributions
from sklearn.metrics import roc_auc_score,average_precision_score
np.set_printoptions(threshold=np.inf)


from google.colab import drive
drive.mount('/content/drive')


W_gene      =  pd.read_table( "/content/drive/My Drive/Data Fusion/GBM_Adjacency_matrices/gene.txt", delim_whitespace=True).to_numpy()
W_methy     =  pd.read_table( "/content/drive/My Drive/Data Fusion/GBM_Adjacency_matrices/methy.txt", delim_whitespace=True).to_numpy()
W_mirna     =  pd.read_table( "/content/drive/My Drive/Data Fusion/GBM_Adjacency_matrices/mirna.txt", delim_whitespace=True).to_numpy()

gene      =  pd.read_table( "/content/drive/My Drive/Data Fusion/GBM/GLIO_Gene_Expression.txt", delim_whitespace=True).to_numpy().transpose()
methy     =  pd.read_table( "/content/drive/My Drive/Data Fusion/GBM/GLIO_Methy_Expression.txt", delim_whitespace=True).to_numpy().transpose()
mirna     =  pd.read_table( "/content/drive/My Drive/Data Fusion/GBM/GLIO_Mirna_Expression.txt", delim_whitespace=True).to_numpy().transpose()
truelabel =  pd.read_table( "/content/drive/My Drive/Data Fusion/GBM/GLIO_Survival.txt", delim_whitespace=True).to_numpy()



print(gene.shape)
print(methy.shape)
print(mirna.shape)

def get_nomralized_laplacian(a):
    b = np.diag(np.power(1.0 / a.sum(axis=1), 0.5))
    return (b @ a @ b)





features = np.c_[mirna,methy,gene]

patient_no = features.shape[0]

idx = np.arange(patient_no)
np.random.shuffle(idx)
train_idx = idx[0:int(patient_no * 0.75)]
val_idx =   idx[int(patient_no * 0.75):]

train_features = features
val_features   = features[val_idx]
print(train_features.shape)
print(val_features.shape)




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.


(215, 12042)
(215, 1305)
(215, 534)
(215, 13881)
(54, 13881)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.


In [0]:
#Model definition Cell

num_features = features.shape[1]

train_features_tensor  = torch.Tensor(train_features)
val_features_tensor    = torch.Tensor(val_features)





class Model(torch.nn.Module):

    def __init__(self,**kwargs):
        super(Model, self).__init__(**kwargs)
        
        
        
            

        self.W0 = torch.nn.Linear(num_features,6000)
        self.W1 = torch.nn.Linear(6000,3000)
        self.W2 = torch.nn.Linear(3000,6000)
        self.W3 = torch.nn.Linear(6000,num_features)

        
        param_list = []
        param_list += list(self.W0.parameters())
        param_list += list(self.W1.parameters())
        param_list += list(self.W2.parameters())
        param_list += list(self.W3.parameters())
        
        

        self.optimizer = torch.optim.Adam(param_list,lr=0.0001)
        self.recon = None
        
        
        

    


        
    def forward(self,x) : 
        self.layer_1 = torch.nn.functional.tanh(self.W0(x))
        self.layer_2 = torch.nn.functional.tanh(self.W1(self.layer_1))
        self.layer_3 = torch.nn.functional.tanh(self.W2(self.layer_2))
        self.layer_4 = self.W3(self.layer_3)
        


        self.x_prim = self.layer_4
        self.embedding = self.layer_2
        self.recon_loss = ((self.x_prim-x)**2).mean(1).mean(0)
        self.loss = self.recon_loss
        
        
        
        
        return self.loss.item()
        
    def backward(self) :
        
        self.optimizer.zero_grad()
        self.loss.backward()
        self.optimizer.step()

        
        
    def get_cosine_sim_emb(self):
        
        emb = self.embedding.detach().numpy()

        norms =  (emb*emb).sum(1).reshape(-1,1)**0.5
        normalized_emb = emb/norms

        emb_matrix = np.abs( normalized_emb@normalized_emb.transpose())
        return emb_matrix
        

    def get_euclidian(self):
        emb = self.embedding.detach().numpy()
        
        emb_matrix = np.zeros((emb.shape[0],emb.shape[0]))
        
        for i in range(emb.shape[0]):
            for j in range(emb.shape[0]):
                emb_matrix[i,j] = ((emb[i]-emb[j])**2).sum()
                
        return emb_matrix
        
        



In [0]:
#Training Cell 

my_model = Model()


for epoch_counter in range(20000):
    
    train_loss = my_model.forward(train_features_tensor)
    my_model.backward()
    #val_loss = my_model.forward(val_features_tensor)
    val_loss = 0
    
    if epoch_counter % 1 ==0:
        print("Epoch:", '%04d' % (epoch_counter + 1), "train_loss=", "{:.5f}".format(train_loss), "val_loss=", "{:.5f}".format(val_loss))


print('\n',"#########################################################",'\n')






/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1374: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Epoch: 0001 train_loss= 1.00520 val_loss= 0.00000
Epoch: 0002 train_loss= 0.94987 val_loss= 0.00000
Epoch: 0003 train_loss= 0.85977 val_loss= 0.00000
Epoch: 0004 train_loss= 0.77515 val_loss= 0.00000
Epoch: 0005 train_loss= 0.71373 val_loss= 0.00000
Epoch: 0006 train_loss= 0.67032 val_loss= 0.00000
Epoch: 0007 train_loss= 0.64502 val_loss= 0.00000
Epoch: 0008 train_loss= 0.62078 val_loss= 0.00000
Epoch: 0009 train_loss= 0.59142 val_loss= 0.00000
Epoch: 0010 train_loss= 0.56947 val_loss= 0.00000
Epoch: 0011 train_loss= 0.55494 val_loss= 0.00000
Epoch: 0012 train_loss= 0.53885 val_loss= 0.00000
Epoch: 0013 train_loss= 0.52266 val_loss= 0.00000
Epoch: 0014 train_loss= 0.50601 val_loss= 0.00000
Epoch: 0015 train_loss= 0.49164 val_loss= 0.00000
Epoch: 0016 train_loss= 0.48041 val_loss= 0.00000
Epoch: 0017 train_loss= 0.47129 val_loss= 0.00000
Epoch: 0018 train_loss= 0.46017 val_loss= 0.00000
Epoch: 0019 train_loss= 0.44966 val_loss= 0.00000
Epoch: 0020 train_loss= 0.44090 val_loss= 0.00000


KeyboardInterrupt: ignored

In [0]:
#my_model.forward(features_tensor)
emb = my_model.embedding.detach().numpy()
        
emb_matrix = np.zeros((emb.shape[0],emb.shape[0]))
        
for i in range(emb.shape[0]):
    for j in range(emb.shape[0]):
        #emb_matrix[i,j] =  ((emb[i]-emb[j])**2).sum()
        emb_matrix[i,j] =  np.exp(-((emb[i]-emb[j])**2).sum()/120)
                
print(emb_matrix)

[[1.00000000e+00 5.47855577e-09 7.29332341e-10 9.61671523e-11
  5.90132408e-11 5.32936999e-12 8.67549485e-13 4.16892764e-11
  1.97902584e-11 9.00567210e-12 2.34703116e-11 2.36242063e-10
  5.85629016e-09 1.56113315e-11 4.65057261e-12 2.38421464e-11
  4.88801800e-08 2.64421965e-10 2.17827753e-10 1.99839968e-10
  6.58699330e-11 1.45877612e-11 4.27697529e-12 4.20541565e-13
  3.58790990e-13 9.31040182e-13 1.64494819e-11 3.71532317e-13
  3.06652248e-11 1.50106442e-11 4.72269377e-12 4.31539548e-13
  3.62122581e-13 3.62482842e-11 4.10847372e-13 2.27741209e-12
  4.85473381e-13 6.57375611e-13 7.36116987e-12 3.86242693e-11
  2.80515427e-12 7.02991031e-12 1.27651052e-11 7.23398844e-13
  3.21786454e-10 4.05221032e-12 1.42282828e-11 1.81245874e-10
  1.17492286e-10 2.81403571e-10 6.77782227e-13 8.40498831e-10
  2.85983580e-09 5.05399610e-12 6.87530700e-10 3.65650034e-11
  5.65964721e-11 2.59135587e-13 5.69820500e-13 7.69749206e-12
  7.59443675e-12 6.47849716e-12 5.75586870e-13 2.33298043e-12
  8.4102

In [0]:
np.savetxt("/content/drive/My Drive/Data Fusion/results/emb_sade.txt",emb,delimiter='\t')
#np.savetxt("/content/drive/My Drive/Data Fusion/gvae_fusion_results/res_sade.txt",emb_matrix,delimiter='\t')

In [0]:
from sklearn.cluster import KMeans
X = emb
n_clusters=3
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(X)

for i in range(n_clusters):
    indx = np.argwhere(kmeans.labels_==i).reshape(-1)
    print(truelabel[indx][:,1].mean())



626.8765432098766
536.3030303030303
573.3168316831683


In [0]:
emb_matrix = my_model.get_cosine_sim_emb()
print(emb_matrix)
np.savetxt("gvae_fusion_results/res_cosine.txt",emb_matrix,delimiter='\t')

[[9.99999762e-01 1.07528433e-01 2.69530844e-02 9.13520753e-02
  1.03215739e-01 7.80488998e-02 5.88160381e-02 2.92040147e-02
  3.22823524e-02 2.40955632e-02 2.25115180e-01 5.83793297e-02
  6.07349798e-02 7.87358359e-02 9.43368226e-02 6.90505430e-02
  2.34133713e-02 4.30251881e-02 3.36033434e-01 3.34922850e-01
  1.67553216e-01 1.98978633e-02 2.84192488e-02 2.92781666e-02
  1.31397210e-02 3.53594646e-02 2.65772641e-01 4.52555791e-02
  4.13791835e-02 2.36687094e-01 9.38652232e-02 1.55723453e-01
  2.88900912e-01 2.28089005e-01 5.03865629e-03 2.44999051e-01
  1.20611086e-01 2.92277858e-02 1.25792027e-01 4.08527851e-02
  2.19282031e-01 1.06328651e-01 9.04962942e-02 2.38926888e-01
  1.54283881e-01 2.10222602e-01 1.37868032e-01 2.08886683e-01
  2.64776915e-01 2.95316100e-01 6.46592751e-02 5.15146069e-02
  4.34365980e-02 1.80428043e-01 1.37596533e-01 1.18417107e-01
  1.36523679e-01 8.89368914e-03 1.69216335e-01 1.62840560e-02
  3.34962867e-02 1.38020709e-01 1.60055198e-02 5.96024096e-02
  1.8452